In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Data/Department/"
tr_train = pd.read_csv(f"{DATA_PATH}trans_train.csv")
target_train = pd.read_csv(f"{DATA_PATH}target_train.csv")

tr_train.shape, target_train.shape

((167465, 7), (2554, 2))

In [ ]:
target_train

,고객번호,target
0,10004,1
1,10013,0
2,10023,0
3,10040,0
4,10052,0
...,...,...
2549,49813,0
2550,49892,0
2551,49925,0
2552,49943,0


In [ ]:
tr_train["구매일자"] = pd.to_datetime(tr_train["구매일자"])
agg_dict = {
    '구매일자':[
        ("내점일수","nunique"),
        ("구매주기",lambda x : (x.max() - x.min()).days / x.nunique() ), # 최근 구매일과 최초 구매일의 차이를 구하고 내점일수로 나누면 구매주기라고 볼수있음
        ("주말방문비율", lambda x : (x.dt.weekday > 4).mean() ), # 4보다 크면 토,일 이니까!!
        ("주구매요일" , lambda x: x.dt.weekday.mode()[0] ), # 최빈값이 여러개 일수도 있어서..
        ("봄구매비율" , lambda x : (x.dt.month.isin([3,4,5])).mean() ),
        ("여름구매비율" , lambda x : (x.dt.month.isin([6,7,8])).mean() ),
        ("가을구매비율" , lambda x : (x.dt.month.isin([9,10,11])).mean() ),
        ("겨울구매비율" , lambda x : (x.dt.month.isin([12,1,2])).mean() ),
        ("일평균구매건수" , lambda x : x.count() / x.nunique() ), # 구매건수를 구매일자로 나누면 일평균 구매건수라고 볼수있음.
        ("거래개월수" , lambda x : x.dt.month.nunique() )
    ],
    '지점':[
         ("방문지점수","nunique"),
         ("주구매지점" , lambda x : x.mode()[0])
    ],
    '상품번호': [
             ('상품번호수', 'nunique'),
             ],
    '중분류': [
            ('중분류수', 'nunique'),
            ('주구매중분류', lambda x: x.mode()[0]),
            ],
    '대분류': [
            ('대분류수', 'nunique'),
            ('주구매대분류', lambda x: x.mode()[0]),
            ],
    '구매가격' : [
        ('총구매액','sum'),
        ('구매건수', 'count'),
        ('평균구매액', "mean"),
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ),
        ('구매금액표준편차',lambda x: x[x>0].std() ),
    ],
}
tmp = tr_train.groupby('고객번호').agg(agg_dict)
tmp.columns = tmp.columns.droplevel()
tmp = tmp.reset_index()
target_train = target_train.merge(tmp,how="left",on = "고객번호")
target_train.head()

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,대분류수,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,25,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,9,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,17,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,15,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,20,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06


In [ ]:
target_train = target_train.fillna(0) # 0으로 채움
target_train.isnull().sum().sum()

0

In [ ]:
target_train = target_train.iloc[:,:-2]
target_train

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,중분류수,주구매중분류,대분류수,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,39,상품군미지정,25,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,12,N.B정장,9,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,23,N/B골프의류,17,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,29,어덜트,15,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,37,시티웨어,20,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,55,상품군미지정,20,가공식품,47428060,371,1.278384e+05,9120000,1000.0,-1532000
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,34,농산가공,20,농산물,54369820,121,4.493374e+05,7500000,1100.0,0
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,2,어덜트,1,명품,1657400,3,5.524667e+05,1015000,39000.0,0
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,8,건과,7,농산물,40742400,11,3.703855e+06,21780000,36000.0,0


In [ ]:
target_train.astype({"평균구매액":"int"})

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,중분류수,주구매중분류,대분류수,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,39,상품군미지정,25,가공식품,390166360,68,5737740,41760000,6570.0,-4700000
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,12,N.B정장,9,가공식품,22447800,17,1320458,13375000,6900.0,0
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,23,N/B골프의류,17,골프,81765810,43,1901530,14325000,1250.0,-6003000
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,29,어덜트,15,명품,213705680,66,3237964,24738000,1500.0,-12948000
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,37,시티웨어,20,시티웨어,358336570,111,3228257,38328000,1250.0,-30868000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,55,상품군미지정,20,가공식품,47428060,371,127838,9120000,1000.0,-1532000
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,34,농산가공,20,농산물,54369820,121,449337,7500000,1100.0,0
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,2,어덜트,1,명품,1657400,3,552466,1015000,39000.0,0
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,8,건과,7,농산물,40742400,11,3703854,21780000,36000.0,0


In [ ]:
# 75% 지점 이상치
np.arange(0,1,0.75)
q_arr = target_train["평균구매액"].quantile([0.75]).to_numpy()
q_arr = q_arr.astype(int)
q_arr

array([2758896])

In [ ]:
def chk_con(x,q_arr=2758895):
    if x >= q_arr:
        return 1
    else:
        return 0

target_train["고객판별"] = target_train["평균구매액"].apply(lambda x : chk_con(x))
target_train

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,주구매중분류,대분류수,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,고객판별
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,상품군미지정,25,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,N.B정장,9,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,N/B골프의류,17,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,0
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,어덜트,15,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,1
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,시티웨어,20,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,상품군미지정,20,가공식품,47428060,371,1.278384e+05,9120000,1000.0,-1532000,0
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,농산가공,20,농산물,54369820,121,4.493374e+05,7500000,1100.0,0,0
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,어덜트,1,명품,1657400,3,5.524667e+05,1015000,39000.0,0,0
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,건과,7,농산물,40742400,11,3.703855e+06,21780000,36000.0,0,1


In [ ]:
agg_list = [
    ("월평균구매빈도", lambda x : x.count() / x.dt.month.nunique() )
]
tmp = tr_train.groupby("고객번호")["구매일자"].agg(agg_list).reset_index()
tmp

,고객번호,월평균구매빈도
0,10004,7.555556
1,10013,3.400000
2,10023,3.909091
3,10040,6.600000
4,10052,12.333333
...,...,...
2549,49813,30.916667
2550,49892,11.000000
2551,49925,1.500000
2552,49943,1.833333


In [ ]:
target_train = target_train.merge(tmp,how="left",on = "고객번호") 
target_train

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,대분류수,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,고객판별,월평균구매빈도
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,25,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,7.555556
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,9,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.400000
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,17,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,0,3.909091
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,15,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,1,6.600000
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,20,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,1,12.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,20,가공식품,47428060,371,1.278384e+05,9120000,1000.0,-1532000,0,30.916667
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,20,농산물,54369820,121,4.493374e+05,7500000,1100.0,0,0,11.000000
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,1,명품,1657400,3,5.524667e+05,1015000,39000.0,0,0,1.500000
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,7,농산물,40742400,11,3.703855e+06,21780000,36000.0,0,1,1.833333


In [ ]:
tmp = tr_train.groupby('고객번호')['구매일자'].max() # 최근 방문 일자
tmp

고객번호
10004   2007-10-31
10013   2007-12-01
10023   2007-12-21
10040   2007-12-11
10052   2007-11-25
           ...    
49813   2007-12-26
49892   2007-11-21
49925   2007-07-26
49943   2007-12-17
49974   2007-11-25
Name: 구매일자, Length: 2554, dtype: datetime64[ns]

In [ ]:
features2 = tmp.copy()
features2 = features2.reset_index()
features2

,고객번호,구매일자
0,10004,2007-10-31
1,10013,2007-12-01
2,10023,2007-12-21
3,10040,2007-12-11
4,10052,2007-11-25
...,...,...
2549,49813,2007-12-26
2550,49892,2007-11-21
2551,49925,2007-07-26
2552,49943,2007-12-17


In [ ]:
avg_day = target_train.groupby('고객번호')['구매주기'].mean()
avg_day.reset_index()

,고객번호,구매주기
0,10004,11.423077
1,10013,21.090909
2,10023,16.666667
3,10040,13.833333
4,10052,8.892857
...,...,...
2549,49813,4.253012
2550,49892,10.096774
2551,49925,18.333333
2552,49943,30.285714


In [ ]:
features2 = pd.merge(features2,avg_day,how="left",on="고객번호")
features2

,고객번호,구매일자,구매주기
0,10004,2007-10-31,11.423077
1,10013,2007-12-01,21.090909
2,10023,2007-12-21,16.666667
3,10040,2007-12-11,13.833333
4,10052,2007-11-25,8.892857
...,...,...,...
2549,49813,2007-12-26,4.253012
2550,49892,2007-11-21,10.096774
2551,49925,2007-07-26,18.333333
2552,49943,2007-12-17,30.285714


In [ ]:
# 2008년 발표날과 최근 구매일의 차이
from datetime import datetime
tr_train["구매일자"] = pd.to_datetime(tr_train["구매일자"])
mk_dt = datetime(2008,1,1)
time_delta = mk_dt - features2["구매일자"]
td = time_delta.dt.days # timedelta 타입 변환

td

0        62
1        31
2        11
3        21
4        37
       ... 
2549      6
2550     41
2551    159
2552     15
2553     37
Name: 구매일자, Length: 2554, dtype: int64

In [ ]:
idx = pd.DataFrame(tr_train["고객번호"].unique(),columns=["고객번호"])
idx = pd.concat([idx,td], axis=1)
x = pd.merge(features2,idx, how="left",on="고객번호")
x

,고객번호,구매일자_x,구매주기,구매일자_y
0,10004,2007-10-31,11.423077,62
1,10013,2007-12-01,21.090909,31
2,10023,2007-12-21,16.666667,11
3,10040,2007-12-11,13.833333,21
4,10052,2007-11-25,8.892857,37
...,...,...,...,...
2549,49813,2007-12-26,4.253012,6
2550,49892,2007-11-21,10.096774,41
2551,49925,2007-07-26,18.333333,159
2552,49943,2007-12-17,30.285714,15


In [ ]:
# 휴면고객인지 아닌지의 판별치
sub1=x["구매주기"]/x["구매일자_y"]
sub1 = pd.DataFrame(sub1)
sub1

,0
0,0.184243
1,0.680352
2,1.515152
3,0.658730
4,0.240347
...,...
2549,0.708835
2550,0.246263
2551,0.115304
2552,2.019048


In [ ]:
x = (sub1 >= 1).astype(int).rename(columns={0:"휴면/이탈 고객 여부"})
x = pd.DataFrame(x)
x

,휴면/이탈 고객 여부
0,0
1,0
2,1
3,0
4,0
...,...
2549,0
2550,0
2551,0
2552,1


In [ ]:
data = pd.concat([target_train,x],axis=1)
data 

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,고객판별,월평균구매빈도,휴면/이탈 고객 여부
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,7.555556,0
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.400000,0
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,0,3.909091,1
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,1,6.600000,0
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,1,12.333333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,가공식품,47428060,371,1.278384e+05,9120000,1000.0,-1532000,0,30.916667,0
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,농산물,54369820,121,4.493374e+05,7500000,1100.0,0,0,11.000000,0
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,명품,1657400,3,5.524667e+05,1015000,39000.0,0,0,1.500000,0
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,농산물,40742400,11,3.703855e+06,21780000,36000.0,0,1,1.833333,1


In [ ]:
data.iloc[:,2:]

,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,...,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,고객판별,월평균구매빈도,휴면/이탈 고객 여부
0,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,...,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,7.555556,0
1,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,...,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.400000,0
2,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,...,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,0,3.909091,1
3,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,...,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,1,6.600000,0
4,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,...,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,1,12.333333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,12,...,가공식품,47428060,371,1.278384e+05,9120000,1000.0,-1532000,0,30.916667,0
2550,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,11,...,농산물,54369820,121,4.493374e+05,7500000,1100.0,0,0,11.000000,0
2551,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,2,...,명품,1657400,3,5.524667e+05,1015000,39000.0,0,0,1.500000,0
2552,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,6,...,농산물,40742400,11,3.703855e+06,21780000,36000.0,0,1,1.833333,1


In [ ]:
cols = target_train.select_dtypes("object").columns
cols

Index(['주구매지점', '주구매중분류', '주구매대분류'], dtype='object')

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit_transform(target_train[cols]).toarray() # 압축된 요소들을 풀어 넘파이 배열로 반환(Compressed Sparse)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
enc.get_feature_names_out() # 원핫 인코딩에 대한 컬럼명들

array(['주구매지점_강남점', '주구매지점_관악점', '주구매지점_광주점', '주구매지점_노원점', '주구매지점_대구점',
       '주구매지점_대전점', '주구매지점_동래점', '주구매지점_미아점', '주구매지점_본  점', '주구매지점_부산본점',
       '주구매지점_부평점', '주구매지점_분당점', '주구매지점_상인점', '주구매지점_센텀시티점', '주구매지점_안양점',
       '주구매지점_영등포점', '주구매지점_울산점', '주구매지점_인천점', '주구매지점_일산점', '주구매지점_잠실점',
       '주구매지점_전주점', '주구매지점_창원점', '주구매지점_청량리점', '주구매지점_포항점',
       '주구매중분류_4대 B/D', '주구매중분류_IT 게임기,S/W', '주구매중분류_KKD', '주구매중분류_L.B',
       '주구매중분류_L/C 아동복', '주구매중분류_L/C골프의류', '주구매중분류_L/C정장', '주구매중분류_MP3 外',
       '주구매중분류_N.B', '주구매중분류_N.B정장', '주구매중분류_N/B 아동복', '주구매중분류_N/B골프의류',
       '주구매중분류_NB 여화', '주구매중분류_TAKE OUT', '주구매중분류_takeout양과',
       '주구매중분류_가공선물세트', '주구매중분류_가공행사', '주구매중분류_가방', '주구매중분류_가스기기',
       '주구매중분류_갈비세트', '주구매중분류_건강가전', '주구매중분류_건강식품', '주구매중분류_건강식품(비타민)',
       '주구매중분류_건강식품(홍삼)', '주구매중분류_건강침대', '주구매중분류_건과', '주구매중분류_건생선',
       '주구매중분류_건어물류', '주구매중분류_계육', '주구매중분류_고정행사(골프)', '주구매중분류_고정행사(스포츠)',
       '주구매중분류_곡물', '주구매중분류_골프용품', '주구매중분류_구두단기행사', '주구매중분류_국산A/V',
       '주구매중분류_

In [ ]:
tmp = pd.DataFrame(
    enc.fit_transform(target_train[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
tmp.head()

,주구매지점_강남점,주구매지점_관악점,주구매지점_광주점,주구매지점_노원점,주구매지점_대구점,주구매지점_대전점,주구매지점_동래점,주구매지점_미아점,주구매지점_본 점,주구매지점_부산본점,...,주구매대분류_주방용품,주구매대분류_차/커피,주구매대분류_축산가공,주구매대분류_침구/수예,주구매대분류_캐주얼,주구매대분류_커리어,주구매대분류_통신/컴퓨터,주구매대분류_트래디셔널,주구매대분류_피혁잡화,주구매대분류_화장품
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data = pd.concat([target_train,tmp], axis=1).drop(columns=cols) # 옆으로 concat하면서 기존에 있던 범주형 컬럼명들은 삭제하기(['주구매지점', '주구매중분류', '주구매대분류'])
data.head()

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,주구매대분류_주방용품,주구매대분류_차/커피,주구매대분류_축산가공,주구매대분류_침구/수예,주구매대분류_캐주얼,주구매대분류_커리어,주구매대분류_통신/컴퓨터,주구매대분류_트래디셔널,주구매대분류_피혁잡화,주구매대분류_화장품
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data.iloc[:,2:]

,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,...,주구매대분류_주방용품,주구매대분류_차/커피,주구매대분류_축산가공,주구매대분류_침구/수예,주구매대분류_캐주얼,주구매대분류_커리어,주구매대분류_통신/컴퓨터,주구매대분류_트래디셔널,주구매대분류_피혁잡화,주구매대분류_화장품
0,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2550,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2551,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2552,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data.columns

Index(['고객번호', 'target', '내점일수', '구매주기', '주말방문비율', '주구매요일', '봄구매비율', '여름구매비율',
       '가을구매비율', '겨울구매비율',
       ...
       '주구매대분류_주방용품', '주구매대분류_차/커피', '주구매대분류_축산가공', '주구매대분류_침구/수예',
       '주구매대분류_캐주얼', '주구매대분류_커리어', '주구매대분류_통신/컴퓨터', '주구매대분류_트래디셔널',
       '주구매대분류_피혁잡화', '주구매대분류_화장품'],
      dtype='object', length=257)

In [ ]:
x_data = data.iloc[:,2:]
y_data = data["target"]

x_data.shape, y_data.shape

((2554, 255), (2554,))

In [ ]:
from sklearn.preprocessing import MinMaxScaler # min-max

scaler = MinMaxScaler() # 스케일러 함수를 받아줄 변수 선언
x_data = scaler.fit_transform(x_data) # 스케일링을 하고자 하는 데이터를 인수로 받음, 리턴값은 numpy 배열형태
x_data

array([[0.08802817, 0.06881372, 0.61764706, ..., 0.        , 0.        ,
        0.        ],
       [0.03521127, 0.12705367, 0.11764706, ..., 0.        , 0.        ,
        0.        ],
       [0.07042254, 0.10040161, 0.34883721, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00704225, 0.11044177, 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.02112676, 0.18244406, 0.54545455, ..., 0.        , 0.        ,
        0.        ],
       [0.01408451, 0.21445783, 0.04761905, ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
SEED = 42
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.2, random_state=SEED)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((2043, 255), (511, 255), (2043,), (511,))

In [ ]:
from sklearn.linear_model import LogisticRegression 
model = LogisticRegression(random_state=SEED) # 모델 객체생성

model.fit(x_train,y_train) # 학습하기(학습,정답)

LogisticRegression(random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score

pred = model.predict(x_valid)
accuracy_score(y_valid, pred)

0.6888454011741683

- 성능 측정 (ROC)

In [ ]:
from sklearn.metrics import roc_auc_score
pred = model.predict_proba(x_valid)[:,1]
roc_auc_score(y_valid,pred)

0.6846428095428496